In [22]:
import networkx as nx
import osmnx as ox
import numpy as np
import pandas as pd

In [41]:
#Get the edge list
edgelist_fn = '../data/processed/BaseEdgeList.csv'
df_edgeList = pd.read_csv(edgelist_fn,
                          header = 1,
                          names = ['From_node','To_node','Site_yield_scf_h'])
#Drop null to values
df_edgeList = df_edgeList.dropna(subset=['To_node'],axis=0)
#Drop zero to values
df_edgeList = df_edgeList[df_edgeList.To_node != 0]
#Change to_nodes to integers then strings
df_edgeList['To_node'] = df_edgeList['To_node'].apply(lambda x: str(int(x)))
#Change from nodes to strings
df_edgeList['From_node'] = df_edgeList['From_node'].apply(lambda x: str(x))

In [44]:
df_edgeList.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2701 entries, 0 to 3110
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   From_node         2701 non-null   object 
 1   To_node           2701 non-null   object 
 2   Site_yield_scf_h  2701 non-null   float64
dtypes: float64(1), object(2)
memory usage: 84.4+ KB


In [45]:
#Create the graph
G = nx.MultiDiGraph()

In [46]:
#Add the edges
for i,r in df_edgeList.iterrows():
    G.add_edge(r.From_node,r.To_node,cost=r.Site_yield_scf_h)

In [49]:
nodeCosts = []
#Iterate through each to_node
for from_node in df_edgeList.From_node.unique():
    #Reset the cost variable
    total_yield = 0
    #Get the upstream nodes for the current node
    up_nodes = nx.ancestors(G,from_node)
    #Append the node itself
    up_nodes.add(from_node)
    #Iterate through the to_node's ancestors
    for up_node in up_nodes:
        #Accumulate the total yield
        total_yield += df_edgeList.query("From_node == '{}'".format(up_node))['Site_yield_scf_h'].sum()
    df_edgeList.loc[df_edgeList.From_node == from_node,'SumYield'] = total_yield

In [50]:
#Compute pipe diameter from throughput
df_edgeList['PipeSize_in'] = df_edgeList['SumYield']**0.394*0.051
#Compute pipe cost from throughput 
df_edgeList['PipeCost'] = df_edgeList['PipeSize_in']**0.485*10829 
#Compute return
df_edgeList['Return_15yr'] = df_edgeList['PipeCost'] / (df_edgeList['SumYield'] * 15)

In [51]:
#Save edge list to a filename
edgeCSV_filename =  '../scratch/Biogas_9May2020.csv'
df_edgeList[['From_node','Site_yield_scf_h', 'SumYield', 'PipeSize_in',
            'PipeCost', 'Return_15yr']].to_csv(edgeCSV_filename,index=False)

In [38]:
%matplotlib inline